<a href="https://colab.research.google.com/github/peterbmob/niteroi/blob/main/2025/DFTB/SK_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab

In [2]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:15
🔁 Restarting kernel...


In [2]:
!conda config --add channels conda-forge
!conda config --set channel_priority strict

In [3]:
!conda install -n base mamba


Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mamba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    c-ares-1.34.5              |       hb9d3cd8_0         202 KB  conda-forge
    ca-certificates-2025.11.12 |       hbd8a1cb_0         149 KB  conda-forge
    certifi-2025.11.12         |     pyhd8ed1ab_0         153 KB  conda-forge
    conda-25.11.0              |  py311h38be061_0         1.2 MB  conda-forge
    conda-libmamba-solver-25.11.0|     pyhd8ed1ab_0          55 KB  conda-forge
    cpp-expected-1.3.1         |       h171cf75_0          24 KB  conda-forge
    fmt-12.0.0                 |       h2b0788b_0         193 KB  conda-forge
    libarchive-3.8.2           | gpl_h7be2006_100         864 KB  conda-forge
    libcurl-8.17.0             |       h4e3cde8_

In [40]:
!mamba install python=3.12 skprogs skprogs-python

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:

  - python=3.12

Pinned packages:

  - python_abi[version="=3.12",build="*cp312*"]

Pinned packages:

  - cuda-version=12


Transaction

  Prefix: /usr/local

  All requested packages already installed


Transaction starting
[+] 0.0s

Transaction finished



In [21]:
!pip install install ase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 137.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 110.9 MB/s eta 0:00:00
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.2.5
    Uninstalling pyparsing-3.2.5:
      Successfully uninstalled pyparsing-3.2.5
  Attempting uninstall: pillow
    Found existing installation: pillow 12.0.0
    Uninstalling pillow-12.0.0:
      Successfully uninstalled pillow-

In [2]:
import ase
print(f"ASE version: {ase.__version__}")

ASE version: 3.26.0


In [29]:
import os
os.chdir('/content')
!rm -rf Parautomatik
!git clone https://github.com/Teoroo-CMC/ParAutomatik.git

fatal: destination path 'ParAutomatik' already exists and is not an empty directory.


In [58]:
'''
Module to generate homo- and hetero-nuclear Slater-Koster files.
'''


import sys
import argparse
import numpy as np


import sktools.common as sc
import sktools.skgen as skgen
from sktools.skdef import Skdef
from sktools import PACKAGE_VERSION


if sys.hexversion < 0x03020000:
    sys.exit('Program only works with Python 3.2 or greater')

if np.__version__.startswith('1.6.'):
    sys.exit('Program only works with Numpy 1.7.x or greater')


SCRIPTNAME = sc.get_script_name()

# Global script logger, will be overriden by the setup_logger() method in
# the respective subcommands depending on the command line loglevel options
logger = None


def main():
    '''Main driver routine.'''

    parser, subparsers = get_parser_and_subparser_container()
    setup_parser_main(parser)
    onecnt_common = get_onecnt_common_parser()
    setup_parser_atom(subparsers, onecnt_common, run_atom)
    setup_parser_denscomp(subparsers, onecnt_common, run_denscomp)
    setup_parser_wavecomp(subparsers, onecnt_common, run_wavecomp)
    twocnt_common = get_twocnt_common_parser()
    setup_parser_twocnt(subparsers, twocnt_common, run_twocnt)
    setup_parser_sktable(subparsers, twocnt_common, run_sktable)
    parse_command_line_and_run_subcommand(parser)


def run_atom(args):
    print('ARGS', args)
    setup_logger(args.loglevel)
    logger.info('Subcommand atom started')
    elements = convert_argument_to_elements(args.element)
    skdefs = merge_skdefs(args.configfiles)
    searchdirs = [args.builddir,] + args.includedirs
    resultdirs = []
    for elem in elements:
        calculator = skgen.run_atom(
            skdefs, elem, args.builddir, searchdirs, args.onecnt_binary,
            args.eigenonly, args.eigenspinonly)
        resultdirs.append(calculator.get_result_directory())
    logger.info('Subcommand atom finished')
    logger.info('Atom results in {}'.format(' '.join(resultdirs)))


def run_denscomp(args):
    setup_logger(args.loglevel)
    logger.info('Subcommand denscomp started')
    elements = convert_argument_to_elements(args.element)
    skdefs = merge_skdefs(args.configfiles)
    searchdirs = [args.builddir,] + args.includedirs
    resultdirs = []
    for elem in elements:
        calculator = skgen.run_denscomp(
            skdefs, elem, args.builddir, searchdirs, args.onecnt_binary)
        resultdirs.append(calculator.get_result_directory())
    logger.info('Subcommand densecomp finished')
    logger.info('Denscomp results in {}'.format(' '.join(resultdirs)))


def run_wavecomp(args):
    setup_logger(args.loglevel)
    logger.info('Subcommand wavecomp started')
    elements = convert_argument_to_elements(args.element)
    skdefs = merge_skdefs(args.configfiles)
    searchdirs = [args.builddir,] + args.includedirs
    resultdirs = []
    for elem in elements:
        calculator = skgen.run_wavecomp(
            skdefs, elem, args.builddir, searchdirs, args.onecnt_binary)
        dirnames = ' '.join(calculator.get_result_directories())
        resultdirs.append(dirnames)
    logger.info('Subcommand wavecomp finished')
    logger.info('Wavecomp results in {}'.format(' '.join(resultdirs)))


def run_twocnt(args):
    print('ARGS', args)
    setup_logger(args.loglevel)
    logger.info('Subcommand twocnt started')
    skdefs = merge_skdefs(args.configfiles)
    builddir = args.builddir
    searchdirs = [builddir,] + args.includedirs
    resultdirs = []
    element_pairs = convert_arguments_to_element_pairs(args.element1,
                                                       args.element2)
    for elem1, elem2 in element_pairs:
        calculator = skgen.run_twocnt(
            skdefs, elem1, elem2, builddir, searchdirs, args.onecnt_binary,
            args.twocnt_binary)
        resultdirs.append(calculator.get_result_directory())
    logger.info('Subcommand twocnt finished')
    logger.info('Twocnt results in {}'.format(' '.join(resultdirs)))


def run_sktable(args):
    setup_logger(args.loglevel)
    logger.info('Subcommand sktable started')
    skdefs = merge_skdefs(args.configfiles)
    builddir = args.builddir
    searchdirs = [builddir,] + args.includedirs
    workdir = args.outdir
    add_dummy_rep = args.dummyrep
    skfiles_written = []
    element_pairs = convert_arguments_to_element_pairs(args.element1,
                                                       args.element2)
    for elem1, elem2 in element_pairs:
        skfiles_written += skgen.run_sktable(
            skdefs, elem1, elem2, builddir, searchdirs, args.onecnt_binary,
            args.twocnt_binary, workdir, add_dummy_rep)
    logger.info('Directory with assembled SK-file(s): {}'.format(workdir))
    logger.info('SK-file(s) written: {}'.format(' '.join(skfiles_written)))


def get_parser_and_subparser_container():
    parser = argparse.ArgumentParser(
        description='General tool for generating Slater-Koster tables.')
    subparsers = parser.add_subparsers(title='available subcommands',
                                       help='')
    return parser, subparsers


def get_onecnt_common_parser():
    '''Common settings for all one-center calculations.'''
    onecnt_common = argparse.ArgumentParser(add_help=False)
    onecnt_common.add_argument(
        'element', help='element to process: either one element (e.g. N) or a '
        'comma separated list of element names *without* spaces in between '
        '(e.g. N,C,H)')
    return onecnt_common


def get_twocnt_common_parser():
    twocnt_common = argparse.ArgumentParser(add_help=False)
    twocnt_common.add_argument(
        'element1', help='first element of the element pair to process: '
        'either one element (e.g. N) or a comma separated list of element '
        'names *without* spaces in between (e.g. N,C,H)')
    twocnt_common.add_argument(
        'element2', help='second element of the element pair to process: '
        'either one element (e.g. N) or a comma separated list of element '
        'names *without* spaces in between (e.g. N,C,H)')
    return twocnt_common


def setup_parser_main(parser):
    parser.add_argument('--version', action='version',
                        version='sktools {}'.format(PACKAGE_VERSION))
    parser.add_argument(
        '-I', '--include-dir', action='append', default=[],
        dest='includedirs',
        help='directory to include in the search for calculation '
        '(default: build directory only)')
    parser.add_argument(
        '-c', '--config-file', action='append', dest='configfiles',
        default=['skdef.hsd',],
        help='config file(s) to be parsed (default: ./skdef.hsd)'
    )
    parser.add_argument(
        '-b', '--build-dir', default='_build', dest='builddir',
        help='build directory (default: _build)')
    parser.add_argument(
        '-o', '--onecenter-binary', dest='onecnt_binary', default=None,
        help='binary to use for the one-center calculations (default: depends '
        'on the calculator specified in the input)')
    parser.add_argument(
        '-t', '--twocenter-binary', dest='twocnt_binary', default=None,
        help='binary to use for the two-center calculationrs (default: depends '
        'on the calculator speciefied in the input)')
    parser.add_argument(
        '-l', '--log-level', dest='loglevel', default='info',
        choices=['debug', 'info', 'warning', 'error'],
        help='Logging level (default: info)')


def setup_parser_atom(subparsers, onecnt_common, target_function):
    parser_atom = subparsers.add_parser(
        'atom', parents=[onecnt_common],
        help='calculates the free atom to get eigenlevels, hubbard values, spin'
        ' couplings, etc.')
    parser_atom.add_argument(
        '-e', '--eigenlevels-only', dest='eigenonly', action='store_true',
        default=False, help='calculates only eigenlevels of the spin '
                            'unpolarized atom but no derivatives.')
    parser_atom.add_argument(
        '-s', '--spin-polarized', dest='eigenspinonly', action='store_true',
        default=False, help='calculates only the eigenlevels of the spin '
                            'polarized atom but no derivatives')
    parser_atom.set_defaults(func=target_function)


def setup_parser_denscomp(subparsers, onecnt_common, target_function):
    parser_denscomp = subparsers.add_parser(
        'denscomp', parents=[onecnt_common],
        help='calculates density compression')
    parser_denscomp.set_defaults(func=target_function)


def setup_parser_wavecomp(subparsers, onecnt_common, target_function):
    parser_wavecomp = subparsers.add_parser(
        'wavecomp', parents=[onecnt_common],
        help='calculates wave function compression')
    parser_wavecomp.set_defaults(func=target_function)


def setup_parser_twocnt(subparsers, twocnt_common, target_function):
    parser_twocnt = subparsers.add_parser(
        'twocnt', parents=[twocnt_common],
        help='calculates two center integrals')
    parser_twocnt.set_defaults(func=target_function)


def setup_parser_sktable(subparsers, twocnt_common, target_function):
    parser_sktable = subparsers.add_parser(
        'sktable', parents=[twocnt_common],
        help='creates an sktable for a given element pair')
    parser_sktable.add_argument(
        '-d', '--dummy-repulsive', action='store_true', dest='dummyrep',
        default=False, help='add dummy repulsive spline to the sk tables')
    parser_sktable.add_argument(
        '-o', '--output-dir', dest='outdir', default='.',
        help='directory where the skfiles should be written to (default: .)')
    parser_sktable.set_defaults(func=target_function)


def parse_command_line_and_run_subcommand(parser):
    args = parser.parse_args()
    print('peppe',args)
    args.func(args)
    print('peppe',args)

def setup_logger(loglevel):
    global logger
    logger = sc.get_script_logger(loglevel, SCRIPTNAME)


def merge_skdefs(filenames):
    '''Returns a merged skdefs object using all specified skdef files.'''

    skdef = Skdef.fromfile(filenames[0])
    for filename in filenames[1:]:
        skdef2 = Skdef.fromfile(filename)
        skdef.update(skdef2)
    return skdef


def convert_argument_to_elements(argument):
    return argument.split(',')


def convert_arguments_to_element_pairs(argument1, argument2):
    elements1 = convert_argument_to_elements(argument1)
    elements2 = convert_argument_to_elements(argument2)
    processed = set()
    element_pairs = []
    for elem1 in elements1:
        elem1low = elem1.lower()
        for elem2 in elements2:
            elem2low = elem2.lower()
            already_processed = ((elem1low, elem2low) in processed
                                 or (elem2low, elem1low) in processed)
            if not already_processed:
                element_pairs.append((elem1, elem2))
                processed.add((elem1low, elem2low))
    return element_pairs

In [74]:
PARAUTOMATIK_PATH='/content/ParAutomatik/'   # Add your installation path here

In [75]:
import os, sys, warnings
import glob
import re
from shutil import copyfile, copytree
import itertools
import numpy as np
from ase.data import atomic_numbers, atomic_names, atomic_masses, covalent_radii, vdw_radii
warnings.filterwarnings('ignore')
import argparse
import numpy as np
import sktools.common as sc
import sktools.skgen as skgen
from sktools.skdef import Skdef
from sktools import PACKAGE_VERSION

base_dir=os.getcwd()  # You should be in your production folder now.
print('Base dir is:', base_dir)
cwd = os.getcwd()
skdef_all = PARAUTOMATIK_PATH+'/pgm/SK_def/skdef.hsd'  # parameter file for the data generation
SCRIPTNAME = sc.get_script_name()

# make the directory for the SK-generation and move in to that directory
if not os.path.isdir("SKF/"):
    os.mkdir("SKF/")

os.chdir("SKF/")
print('You are here',os.getcwd())

Base dir is: /content/SKF/SKF
You are here /content/SKF/SKF/SKF


In [79]:
# automatic search for elements in database
print('You are here',os.getcwd())

import ase.db as db
DFT_DB=db.connect("/content/ParAutomatik/Electrolyte/DFT_DB.db")
elements=[]
for row in DFT_DB.select():
    structure=row.toatoms()
    a=structure.get_chemical_symbols()
    for i in a:
        if i not in elements:
            elements.append(i)

elem=''
for i in elements:
    if elem=='':
        elem=i
    else:
        elem=elem+','+i
print(elem)

You are here /content/SKF/SKF/SKF
C,H,Li,O


In [ ]:
#set up input for skprogs and run skprogs generating the Slater-Koster files with a dummy rep
parser, subparsers = get_parser_and_subparser_container()
setup_parser_main(parser)
onecnt_common = get_onecnt_common_parser()
setup_parser_atom(subparsers, onecnt_common, run_atom)
setup_parser_denscomp(subparsers, onecnt_common, run_denscomp)
setup_parser_wavecomp(subparsers, onecnt_common, run_wavecomp)
twocnt_common = get_twocnt_common_parser()
setup_parser_twocnt(subparsers, twocnt_common, run_twocnt)
setup_parser_sktable(subparsers, twocnt_common, run_sktable)

class Args:
    includedirs=[]
    configfiles=[skdef_all]
    builddir='_build'
    onecnt_binary='slateratom'
    twocnt_binary='sktwocnt'
    loglevel='info'
    element1=elem
    element2=elem
    dummyrep=True
    outdir='.'
    func=run_sktable


args=Args()

#run
args.func()

INFO:colab_kernel_launcher.py:Subcommand sktable started
[colab_kernel_launcher.py] Subcommand sktable started
[colab_kernel_launcher.py] Subcommand sktable started
[colab_kernel_launcher.py] Subcommand sktable started
[colab_kernel_launcher.py] Subcommand sktable started
[colab_kernel_launcher.py] Subcommand sktable started
INFO:skgen.sktable:Started for C-C
[skgen.sktable] Started for C-C
[skgen.sktable] Started for C-C
[skgen.sktable] Started for C-C
[skgen.sktable] Started for C-C
[skgen.sktable] Started for C-C
INFO:skgen.sktable:Creating free atom prerequisite for C
[skgen.sktable] Creating free atom prerequisite for C
[skgen.sktable] Creating free atom prerequisite for C
[skgen.sktable] Creating free atom prerequisite for C
[skgen.sktable] Creating free atom prerequisite for C
[skgen.sktable] Creating free atom prerequisite for C
INFO:skgen.atom:Started for C
[skgen.atom] Started for C
[skgen.atom] Started for C
[skgen.atom] Started for C
[skgen.atom] Started for C
[skgen.atom] 

condacolab_install.log	DFT_DB.db  ParAutomatik  sample_data  SKF
